In [0]:
PFSSPEC_ROOT = '/home/dobos/project/pfsspec-all/'
PFSSPEC_DATA = '/datascope/subaru/data/pfsspec'
GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/bosz/bosz_5000_full'

In [0]:
import os, sys, warnings

# Allow load project as module
sys.path.insert(0, PFSSPEC_ROOT)

# Set environment
os.environ['PFSSPEC_ROOT'] = PFSSPEC_ROOT
os.environ['PFSSPEC_DATA'] = PFSSPEC_DATA
os.environ['PYSYN_CDBS'] = os.path.join(os.environ['PFSSPEC_DATA'], 'cdbs')

# Filter warnings
warnings.filterwarnings("ignore")

In [0]:
%pylab inline

In [0]:
import os
import getpass
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

from scipy.optimize import curve_fit
from scipy import ndimage
from scipy.ndimage.filters import maximum_filter1d
from scipy.interpolate import Rbf, interp1d, interpn

from pfsspec.core.physics import Physics

from pfsspec.stellar.continuum.model.alex import Alex, AlexContinuumModelTrace

# Load grid

In [0]:
from pfsspec.core.grid import ArrayGrid
from pfsspec.stellar.grid import ModelGrid
from pfsspec.stellar.grid.bosz import Bosz

In [0]:
fn = os.path.join(os.environ['PFSSPEC_DATA'], GRID_PATH, 'spectra.h5')

In [0]:
grid = ModelGrid(Bosz(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

In [0]:
for k in grid.grid.axes:
    print(k, grid.grid.axes[k].values)

In [0]:
for k in grid.grid.values:
    print(k, grid.grid.value_shapes[k])

# Pick a spectrum and fit with model

In [0]:
M_H = -0.5
T_eff = 3750
log_g = 1
a_M = 0
C_M = 0

In [0]:
idx = grid.array_grid.get_nearest_index(M_H=M_H, T_eff=T_eff, log_g=log_g, a_M=a_M, C_M=C_M)
idx

In [0]:
#idx = (0, 12, 10, 3, 1)

In [0]:
spec = grid.get_model_at(idx)

In [0]:
spec.M_H, spec.T_eff, spec.log_g, spec.a_M, spec.C_M

In [0]:
trace = AlexContinuumModelTrace()
model = Alex(trace=trace)
model.init_wave(spec.wave)

params = model.fit(spec)

In [0]:
for k in params:
    print(k, params[k])

In [0]:
trace.blended_p0, trace.blended_params

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave, np.log(spec.flux), lw=0.1)
plt.plot(spec.wave, np.log(spec.cont), 'b--')

plt.xlim(2800, 15000)
plt.ylim(6, None)
plt.grid(True)

plt.xscale('log')
plt.title('Original spectrum and theoretical continuum')

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave, np.log(spec.flux) - np.log(spec.cont), lw=0.1)
plt.axhline(0, c='b', ls='--')

plt.xlim(2800, 15000)
# plt.ylim(6, None)
plt.grid(True)

plt.xscale('log')
plt.title('Spectrum normalized by theoretical continuum')

In [0]:
wave = model.wave
cont = model.eval_continuum_all(params)

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave, np.log(spec.flux), lw=0.1)
plt.plot(wave, cont, c='r')
plt.plot(spec.wave, np.log(spec.cont), 'b--')

plt.xlim(2800, 15000)
plt.ylim(6, None)
plt.grid(True)

plt.xscale('log')
plt.title('Spectrum and fitted theoretical continuum')

In [0]:
wave = model.wave
blend = model.eval_blended_all(params)

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave[model.wave_mask], np.log(spec.flux[model.wave_mask]) - cont, lw=0.1)
plt.plot(wave, blend, c='r')

for k, v in trace.blended_control_points.items():
    (x, y) = v
    plt.plot(np.exp(x), y, 'r.')

plt.xlim(2800, 15000)
plt.ylim(-0.4, 0.1)
plt.grid(True)

plt.xscale('log')

plt.title('Upper envelope fit')

In [0]:
trace.blended_chi2

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave[model.wave_mask], np.log(spec.flux[model.wave_mask]) - cont, lw=0.1)
plt.plot(wave, blend, c='r')

for k, v in trace.blended_control_points.items():
    (x, y) = v
    plt.plot(np.exp(x), y, 'r.')

plt.xlim(3000, 3500)
plt.ylim(-1, 0.1)
plt.grid(True)

#plt.xscale('log')

plt.title('Upper envelope fit - Blue')

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave[model.wave_mask], np.log(spec.flux[model.wave_mask]) - cont, lw=0.1)
plt.plot(wave, blend, c='r')

for k, v in trace.blended_control_points.items():
    (x, y) = v
    plt.plot(np.exp(x), y, 'r.')

plt.xlim(3500, 6000)
plt.ylim(-0.4, 0.1)
plt.grid(True)

#plt.xscale('log')

plt.title('Upper envelope fit - Balmer')

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave[model.wave_mask], np.log(spec.flux[model.wave_mask]) - cont, lw=0.1)
plt.plot(wave, blend, c='r')

for k, v in trace.blended_control_points.items():
    (x, y) = v
    plt.plot(np.exp(x), y, 'r.')

plt.xlim(8000, 10500)
plt.ylim(-0.005, 0.002)
plt.grid(True)

#plt.xscale('log')

plt.title('Upper envelope fit - Paschen')

In [0]:
wave, cont = model.eval(params)

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(w, c='k')

for w in model.blended_bounds:
    plt.axvline(w, c='green')

plt.plot(spec.wave, np.log(spec.flux), lw=0.1)
plt.plot(wave, cont, c='r')
plt.plot(spec.wave, np.log(spec.cont), 'b--')

plt.xlim(2800, 15000)
plt.ylim(12, None)
plt.grid(True)

plt.xscale('log')

plt.title('Upper envelope fit')

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(np.log(w), c='k')

for w in model.blended_bounds:
    plt.axvline(np.log(w), c='green')

#plt.plot(np.log(spec.wave), np.log(spec.flux / spec.cont))
plt.plot(model.log_wave, np.log(spec.flux[model.wave_mask]) - cont, lw=0.1)

plt.xlim(np.log(2800), np.log(15000))
plt.ylim(-3, 0.1)
plt.grid(True)

plt.title('Normalized spectrum')

In [0]:
plt.figure(figsize=(12, 6))

for w in model.limit_wave:
    plt.axvline(np.log(w), c='k')

for w in model.blended_bounds:
    plt.axvline(np.log(w), c='green')

#plt.plot(np.log(spec.wave), np.log(spec.flux / spec.cont))
plt.plot(model.log_wave, np.log(spec.flux[model.wave_mask]) - cont, lw=0.1)
plt.axhline(0, c='r')

plt.xlim(np.log(2800), np.log(15000))
plt.ylim(-0.05, 0.02)
plt.grid(True)

plt.title('Normalized spectrum')